# 03_ndmi_cdmx_filtering_alcaldias

This notebook loads hexs with vegetation (NDVI) to plot NDMI values january-may for each year __in CDMX.__

Note: Database has only data up until 2023. Data for __ene, feb, mar, aprl 2024__ (current date) was run using Script 17 and saved locally.
*     band_name_dict = {'nir':[True],'swir16':[False], 'eq':["(nir-swir16)/(nir+swir16)"]}
*     start_date = '2024-01-01'
*     end_date = '2024-03-19'
*     save = False
*     local_save = True

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [2]:
city = 'CDMX'

## 1. Load hexs with vegetation in alcaldías of interest

### NDVI Load data - for city and res

NDVI's hex were processed using the correct urban+rural hexs configuration on Script 21.

In [3]:
# Downloading NDVI res 10 data (aprox. 150-200s)
ndvi_schema = 'raster_analysis'
ndvi_table = 'ndvi_analysis_hex'
res = 10
query = f"SELECT * FROM {ndvi_schema}.{ndvi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\" = {res}"
ndvi_res10 = aup.gdf_from_query(query, geometry_col='geometry')

# Shorten data
ndvi_res10 = ndvi_res10[['hex_id','ndvi_median']]

# Show
print(ndvi_res10.shape)
ndvi_res10.head(2)

(59557, 2)


,hex_id,ndvi_median
0,8a4995b8cb57fff,0.10846
1,8a4995b88b4ffff,0.24483


### NDVI Data treatment - Create vegetation categories

In [4]:
# Create vegetation categories
ndvi_res10['ndvi_median_rango'] = np.nan
ndvi_res10.loc[ndvi_res10.ndvi_median>=0.6 , 'ndvi_median_rango'] = 'Alta densidad vegetal'
ndvi_res10.loc[(ndvi_res10.ndvi_median>=0.4 )&
                (ndvi_res10.ndvi_median<0.6), 'ndvi_median_rango'] = 'Moderada densidad vegetal'
ndvi_res10.loc[(ndvi_res10.ndvi_median>=0.2)&
                (ndvi_res10.ndvi_median<0.4), 'ndvi_median_rango'] = 'Mínima densidad vegetal'
ndvi_res10.loc[(ndvi_res10.ndvi_median>=0.1)&
                (ndvi_res10.ndvi_median<0.2), 'ndvi_median_rango'] = 'Suelo'
ndvi_res10.loc[(ndvi_res10.ndvi_median<0.1), 'ndvi_median_rango'] = 'Suelo artificial/Agua/Piedra'

# Ordering data
categories = ['Suelo artificial/Agua/Piedra', 'Suelo', 'Mínima densidad vegetal', 'Moderada densidad vegetal', 'Alta densidad vegetal']
ndvi_res10['ndvi_median_rango'] = pd.Categorical(ndvi_res10['ndvi_median_rango'], categories=categories, ordered=True)

# Show
print(ndvi_res10.shape)
ndvi_res10.head(2)

(59557, 3)


,hex_id,ndvi_median,ndvi_median_rango
0,8a4995b8cb57fff,0.10846,Suelo
1,8a4995b88b4ffff,0.24483,Mínima densidad vegetal


### NDVI Data treatment - Filter for places with vegetation

In [5]:
#Keeping High-moderate-minimal vegetation density
data_withveg = ndvi_res10.loc[(ndvi_res10.ndvi_median_rango == 'Alta densidad vegetal') |
               (ndvi_res10.ndvi_median_rango == 'Moderada densidad vegetal')|
               (ndvi_res10.ndvi_median_rango == 'Mínima densidad vegetal')].copy()

# Show
print(data_withveg.shape)
data_withveg.head(2)

(35317, 3)


,hex_id,ndvi_median,ndvi_median_rango
1,8a4995b88b4ffff,0.244830,Mínima densidad vegetal
4,8a4995b8881ffff,0.259619,Mínima densidad vegetal


### hex_gdf Load data - (Contiene alcaldías)

In [6]:
# Download hex_gdf  (aprox. 150-200s)
hex_schema = 'hexgrid'
hex_table = 'hexgrid_10_city_2020'
query = f"SELECT * FROM {hex_schema}.{hex_table} WHERE \"city\" LIKE \'{city}\'"
hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Shorten data
hex_gdf_f = hex_gdf[['hex_id_10','NOMGEO','geometry']].copy()
hex_gdf_f.rename(columns={'hex_id_10':'hex_id'},inplace=True)

# Show
print(hex_gdf_f.shape)
hex_gdf_f.head(2)

(86349, 3)


,hex_id,NOMGEO,geometry
0,8a4995b8cb57fff,Azcapotzalco,"POLYGON ((-99.19358 19.48193, -99.19293 19.482..."
1,8a4995b88d67fff,Azcapotzalco,"POLYGON ((-99.17778 19.49754, -99.17713 19.497..."


### Data with veg (NDVI) + alcaldías (hex_gdf)

In [7]:
# Inner join to keep data with veg of alcaldías of interest only
data_withveg_alcaldias = data_withveg.merge(hex_gdf_f,on='hex_id',how='inner')

# Show
print(data_withveg_alcaldias.shape)
data_withveg_alcaldias.head(2)

(35317, 5)


,hex_id,ndvi_median,ndvi_median_rango,NOMGEO,geometry
0,8a4995b88b4ffff,0.244830,Mínima densidad vegetal,Azcapotzalco,"POLYGON ((-99.19260 19.50640, -99.19195 19.506..."
1,8a4995b8881ffff,0.259619,Mínima densidad vegetal,Azcapotzalco,"POLYGON ((-99.18272 19.50049, -99.18207 19.500..."


## 2. Load NDMI

### Load NDMI db data (using city, res and tupple of months of analysis)

In [8]:
# Complete dataset NDMI res 10 data 
complete_ndmi_schema = 'raster_analysis'
complete_ndmi_table = 'ndmi_complete_dataset_hex'
res = 10

# OPTION A - Downloading complete dataset and then filtering for months of interest (aprox. 700-1000s)
#query = f"SELECT * FROM {complete_ndmi_schema}.{complete_ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res}"
#complete_ndmi_res10 = aup.df_from_query(query)
#months_ofinterest = [1,2,3,4,5]
#complete_ndmi_res10_f = complete_ndmi_res10.loc[complete_ndmi_res10.month.isin(months_ofinterest)]

# OPTION B - Using hex_lsts and quering with tupple (First time 3494s, second time 3409s)
#hexs_lst = list(data_withveg_alcaldias.hex_id.unique())
#query = f"SELECT * FROM {complete_ndmi_schema}.{complete_ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res} AND \"hex_id\" IN {str(tuple(hexs_lst))}"
#complete_ndmi_res10 = aup.df_from_query(query)

# OPTION C - Downloading complete dataset filtering for months of interest in query (806s)
months_ofinterest = [1,2,3,4,5]
query = f"SELECT * FROM {complete_ndmi_schema}.{complete_ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res} AND \"month\" IN {str(tuple(months_ofinterest))}"
complete_ndmi_res10 = aup.df_from_query(query)

# Show
print(complete_ndmi_res10.shape)
complete_ndmi_res10.head(2)

(1902135, 6)


,hex_id,res,ndmi,month,year,city
0,8a499595944ffff,10,-0.090000,1,2018,CDMX
1,8a4995862907fff,10,0.003396,1,2018,CDMX


### Load NDMI local data (2024)

In [9]:
# Load NDMI local data (2024)
y2024_ndmi_res10 = pd.read_csv("../../../data/processed/tmp_ndmi/local_save/CDMX_ndmi_HexRes10_v0.csv")

# Merge with db data (2018-2023)
complete_ndmi_res10_concat = pd.concat([complete_ndmi_res10,y2024_ndmi_res10])

# Shorten data
complete_ndmi_res10_f = complete_ndmi_res10_concat[['hex_id','ndmi','month','year']]

# Show
print(complete_ndmi_res10_f.shape)
complete_ndmi_res10_f.head(2)

(2140363, 4)


,hex_id,ndmi,month,year
0,8a499595944ffff,-0.090000,1,2018
1,8a4995862907fff,0.003396,1,2018


## Data treatment

### Data treatment - Merge ndmi for places with Data with veg (NDVI) + alcaldías (hex_gdf)

In [12]:
# Inner merge to keep ndmi data for hexs with vegetation only
ndvi_ndmi_gdf = data_withveg_alcaldias.merge(complete_ndmi_res10_f, on='hex_id',how='inner')

# Show
print(ndvi_ndmi_gdf.shape)
ndvi_ndmi_gdf.head(2)

(1200778, 8)


,hex_id,ndvi_median,ndvi_median_rango,NOMGEO,geometry,ndmi,month,year
0,8a4995b88b4ffff,0.24483,Mínima densidad vegetal,Azcapotzalco,"POLYGON ((-99.19260 19.50640, -99.19195 19.506...",-0.000792,1,2018
1,8a4995b88b4ffff,0.24483,Mínima densidad vegetal,Azcapotzalco,"POLYGON ((-99.19260 19.50640, -99.19195 19.506...",-0.014421,2,2018


## Data analysis

### Método elegido - Group by + transform data

In [13]:
# Isolate categories and data
data_analysis = ndvi_ndmi_gdf[['hex_id','year','ndmi']].copy()

# Group by
grouped_by = data_analysis.groupby(['hex_id','year']).mean()
grouped_by.reset_index(inplace=True)

# Transform data
# GDF with unique rows for each geometry (hex)
mean_ndmi_bydate = hex_gdf_f.copy()

# Loop that transforms data to columns
for year in grouped_by.year.unique():
    
    # Isolate new column data (In this case, year)
    idx = grouped_by.year == year
    df_tmp = grouped_by.loc[idx,['hex_id','ndmi']].copy()
    
    # Assign as column name (In this case, year)
    if year == 2024:
        df_tmp = df_tmp.rename(columns={'ndmi':f"ene-abr_{year}"})
    else:
        df_tmp = df_tmp.rename(columns={'ndmi':f"ene-may_{year}"})
    
    # Merge to gdf with unique geometry IDs (In this case, hex_id)
    mean_ndmi_bydate = mean_ndmi_bydate.merge(df_tmp, on='hex_id',how='inner')

mean_ndmi_bydate

,hex_id,NOMGEO,geometry,ene-may_2018,ene-may_2019,ene-may_2020,ene-may_2021,ene-may_2022,ene-may_2023,ene-abr_2024
0,8a4995b88d67fff,Azcapotzalco,"POLYGON ((-99.17778 19.49754, -99.17713 19.497...",0.105919,0.147756,0.087584,0.035289,0.086562,0.028519,0.033012
1,8a4995b81af7fff,Azcapotzalco,"POLYGON ((-99.17055 19.47147, -99.16990 19.471...",0.037642,0.031787,0.044080,0.040128,0.033321,0.029482,0.032508
2,8a4995a36c9ffff,Azcapotzalco,"POLYGON ((-99.21599 19.48356, -99.21535 19.483...",0.005989,-0.005167,-0.011358,-0.020786,0.002482,-0.007598,-0.006222
3,8a4995b8d1a7fff,Azcapotzalco,"POLYGON ((-99.19989 19.49476, -99.19924 19.495...",0.104418,0.105524,0.125967,0.104106,0.086681,0.073300,0.091176
4,8a4995b818a7fff,Azcapotzalco,"POLYGON ((-99.16380 19.46200, -99.16316 19.462...",0.017313,0.015597,0.027663,0.025225,0.015952,0.011709,0.021039
...,...,...,...,...,...,...,...,...,...,...
35312,8a4995b946b7fff,Venustiano Carranza,"POLYGON ((-99.07150 19.44567, -99.07085 19.446...",0.113010,0.108066,0.080905,0.031761,0.079986,0.048547,0.052662
35313,8a4995b96d2ffff,Venustiano Carranza,"POLYGON ((-99.05327 19.43595, -99.05263 19.436...",-0.010727,-0.034766,-0.071402,-0.097678,-0.039400,-0.079977,-0.051626
35314,8a4995bb2017fff,Venustiano Carranza,"POLYGON ((-99.08144 19.41388, -99.08080 19.414...",0.020229,0.009981,0.027224,0.010948,0.017089,0.005639,0.009055
35315,8a4995bb231ffff,Venustiano Carranza,"POLYGON ((-99.08787 19.41894, -99.08722 19.419...",-0.022538,-0.023175,0.006531,0.028634,0.019975,0.005070,0.011997


### Método 1 - 1425s

In [12]:
metodo_1 = False

In [13]:
if metodo_1:
    mean_ndmi_bydate = hex_gdf_f.copy()
    
    i = 0
    for year in data_withveg_alcaldias.year.unique():
        for month in data_withveg_alcaldias.month.unique():
            for alcaldia in data_withveg_alcaldias.NOMGEO.unique():
    
                print(f"Calculating mean ndmi in {alcaldia} date {month}/{year}.")
                    
                data_df = data_withveg_alcaldias.loc[(data_withveg_alcaldias.year == year) &
                                                     (data_withveg_alcaldias.month == month) &
                                                     (data_withveg_alcaldias.NOMGEO == alcaldia)].copy()
    
                for hex_id in data_df['hex_id'].unique():
                    # Get each mean hex_id data for current year, month and alcaldía
                    hex_data = data_df.loc[data_df['hex_id'] == hex_id].copy()
                    mean_ndmi = hex_data['ndmi'].mean()
                    # Register data
                    idx = mean_ndmi_bydate['hex_id'] == hex_id
                    mean_ndmi_bydate.loc[idx,f"{year}_{month}"] = mean_ndmi
    
    # Show
    print(mean_ndmi_bydate.shape)
    mean_ndmi_bydate.head(2)

### Método 2 - Transform data to columns and calculate mean on row axis (Canceled)

In [14]:
metodo_2 = False

In [15]:
if metodo_2:
    data_analysis = data_withveg_alcaldias.copy()
    # Create date column
    data_analysis['year'] = data_analysis['year'].astype('str')
    data_analysis['month'] = data_analysis['month'].astype('str') 
    data_analysis['date'] =  data_analysis['year'] + "_"+ data_analysis['month']
    # Drop columns no longer in use
    data_analysis.drop(columns=['ndvi_median','ndvi_median_rango','month','year'],inplace=True)
    
    # Show
    print(data_analysis.shape)
    data_analysis.head(2)

In [16]:
if metodo_2:
    # TRANSFORM DATA FROM MULTIPLE ROWS TO COLUMNS
    ######################################################################### CÓDIGO BASE PARA REPOSICIONAR DATOS (Script 15-15min-cities)
    # delete duplicates and keep only one point for each node
    #nodes_geom = nodes.drop_duplicates(subset='osmid', keep="last")[['osmid','geometry','metropolis']].copy()
    #nodes_analysis = nodes_geom.copy()
    
    # relate time data to each point
    #for amenidad in list(nodes.amenity.unique()):
    
    #    nodes_tmp = nodes.loc[nodes.amenity == amenidad,['osmid','time']]
    #    nodes_tmp = nodes_tmp.rename(columns={'time':amenidad})
    
    #    if nodes_tmp[amenidad].mean() == 0:
    #        nodes_tmp[amenidad] = np.nan
    
    #    nodes_analysis = nodes_analysis.merge(nodes_tmp, on='osmid')
    
    ######################################################################### PARA ESTE CÓDIGO
    #nodes_analysis (gdf con 1 row para cada geometría) --> proviene de data_analysis 
    #amenidad --> date (lo que serán las columnas en el nuevo gdf)
    #osmid --> hex_id (columna de ID de las geometrías)
    #time --> ndmi (columna con datos a reposicionar)
    
    ######################################################################### APLICACIÓN
    # GDF with unique rows for each geometry
    mean_ndmi_bydate = data_analysis.drop_duplicates(subset='hex_id', keep='last')[['hex_id','geometry','NOMGEO']].copy()
    
    # Loop that transforms data
    for date in data_analysis.date.unique():
        
        # Isolate new column data (In this case, date)
        idx = data_analysis.date == date
        df_tmp = data_analysis.loc[idx,['hex_id','ndmi']].copy()
        
        # Assign as column name (In this case, date)
        df_tmp = df_tmp.rename(columns={'ndmi':date})
        
        # Merge to gdf with unique geometry IDs (In this case, hex_id)
        mean_ndmi_bydate = mean_ndmi_bydate.merge(df_tmp, on='hex_id',how='inner')
    
    mean_ndmi_bydate

## Final processing (Canceled, categorizing NDMI from NDVI values is too complex)

NDMI Interpretation
* -1 – -0.8 Bare soil,
* -0.8 – -0.6 Almost absent canopy cover,
* -0.6 – -0.4 Very low canopy cover,
* -0.4 – -0.2 Low canopy cover, dry or very low canopy cover, wet,
* -0.2 – 0 Mid-low canopy cover, high water stress or low canopy cover, low water stress,
* 0 – 0.2 Average canopy cover, high water stress or mid-low canopy cover, low water stress,
* 0.2 – 0.4 Mid-high canopy cover, high water stress or average canopy cover, low water stress,
* 0.4 – 0.6 High canopy cover, no water stress,
* 0.6 – 0.8 Very high canopy cover, no water stress,
* 0.8 – 1 Total canopy cover, no water stress/waterlogging

Current output_gdf canopy cover will be interpreted the following way:
* 'Mínima densidad vegetal' --> Almost absent canopy cover, very low canopy cover, low canopy cover
* 'Moderada densidad vegetal' --> Mid-low canopy cover, Average canopy cover, Mid-high canopy cover
* ' Alta densidad vegetal' --> High canopy cover, very high canopy cover, Total canopy cover

### [Canceled] Create NDMI Categories - Merge NDVI categories data

In [41]:
#output_gdf = mean_ndmi_bydate.merge(data_withveg_alcaldias[['hex_id','ndvi_median_rango']],on='hex_id',how='inner')

# Show
#print(output_gdf.shape)
#output_gdf.head(2)

In [42]:
#analysis_cols = list(mean_ndmi_bydate.columns)
#analysis_cols.remove('hex_id')
#analysis_cols.remove('NOMGEO')
#analysis_cols.remove('geometry')
#analysis_cols

In [43]:
# Test apply function
#test_gdf = output_gdf.copy()

### Create NDMI Categories b) apply function according to each value.
#def splitname(row):
#    row['First']=row['ndvi_median_rango'].split(" ")[0]
#    row['Second']=row['ndvi_median_rango'].split(" ")[1]
#    row['Last']=row['ndvi_median_rango'].split(" ")[2]
#    return row
    
#Aplicar la función
#test_gdf = test_gdf.apply(splitname, axis='columns')
#test_gdf

In [44]:
# Apply function that categorizes NDMI
#def categorize_ndmi(row):
#    if row['ndvi_median_rango'] == 'Mínima densidad vegetal':
#        if row[ndmi_col] < -0.4:
#            row[f"{ndmi_col}_cat"] = 'Almost absent or very low canopy cover"       

## Save

In [14]:
save_dir = f'../../../data/external/temporal_fromjupyter/'
mean_ndmi_bydate.to_file(save_dir+"cutzamala/cdmx_enemay_ndmi.gpkg")